
# 2D GRAPH-BASED SLAM #

`Goal` : Estimate the state x which best explains the measurements(z).

`x` -> State vector that contains robot poses and landmark positions. It is concatenation of the poses and landmarks of the graph. Ex.( $ {x}_{0}, {y}_{0}, {theta}_{0}, {x}_{l},{y}_{l} ... $ )

`Error` -> ${e}_{i} = {z}_{i} - {f}_{i}{(x)}  = {z}_{i} - \hat{z}_{i}$

`Least Square Term` -> $ {e}_{i}{(x)} = {e}_{i}{(x)}^T  {Ω}_{i} {e}_{i}{(x)}$

`Least Squares Optimization` -> ${x}^* = {argmin}_{x} \sum_{k} {e}_{k}{(x)}^T  {Ω}_{k} {e}_{k}{(x)} $


`Ω` :

- Observation are affected by noise.

- Information matrix ${Ω}_{ij}$ for each edge the encode its uncertainty. 

- The "bigger" ${Ω}_{ij}$ the more the edge "matters" in the optimization.


`Error function for pose to pose:`  ${e}_{ij} ({x}_{i},{x}_{j}) = $ t2v$({Z}_{ij}^{-1} ({X}_{i}^{-1} {X}_{j}))$


- ${Z}_{ij}^{-1}$ = measurement (How ${x}_{j}$ is seen by ${x}_{i}$)

- ${X}_{i}^{-1} {X}_{j}$ = ${x}_{i}$ referenced w.r.t ${x}_{j}$ (How ${x}_{j}$ is seen by ${x}_{i}$)

- t2v, v2t -> forward, backward mapping between homegenous coordinate and x, y, theta.

-  ${Z}_{ij} = {X}_{i}^{-1} {X}_{j} $ -> error will be zero.

`Error function for landmark to pose:` ${e}_{ij} ({x}_{i},{x}_{j}) = \hat{z}_{ij} - {z}_{ij} = {R}_{i}^{T}({x}_{j} - {t}_{i}) - {z}_{ij}$
 
  - ${x}_{i} $ -> robot pose - 3D

  - ${x}_{j} $ -> landmark pose - 2D

  - ${t}_{i} $ -> robot translation vector




`Nonlinear least squares optimization (Gauss Newton)` :

Iterate until convergence ;

- Linearize around x and compute for each measurement 
 - ${e}_{i}{(x+dx)} =~ {e}_{i}{(x)} + {J}_{i}{(dx)} $
- Compute the terms for the linear system 
 - ${b}^{T} = \sum_{i} {e}_{i}^T  {Ω}_{i} {J}_{i}$ 
 - ${H} = \sum_{i} {J}_{i}^T  {Ω}_{i} {J}_{i}$
- Solve the linear system
 - ${dx}^{*} = -{H}^{-1}{b}$
- Update State
 - ${x} = {x} + {dx}$

`Building the Linear System` :

For each constraint ;
- Compute error
 - ${e}_{ij} = {e}({x}_{i},{x}_{j})$
- Compute the blocks of the Jacobian
 - $ {A}_{ij} = \frac{\partial {e}({x}_{i},{x}_{j})}{\partial {x}_{i}}$
 - $ {B}_{ij} = \frac{\partial {e}({x}_{i},{x}_{j})}{\partial {x}_{j}}$
- Update the coefficient vector
 - $\bar{b}_{i}^{T} {+}{=} {e}_{ij}^{T} {Ω}_{ij} {A}_{ij}$ 
 - $\bar{b}_{j}^{T} {+}{=} {e}_{ij}^{T} {Ω}_{ij} {B}_{ij}$
- Update the system matrix
 - $\bar{H}_{ii} {+}{=} {A}_{ij}^{T} {Ω}_{ij} {A}_{ij}$ 
 - $\bar{H}_{ij} {+}{=} {A}_{ij}^{T} {Ω}_{ij} {B}_{ij}$
 - $\bar{H}_{ji} {+}{=} {B}_{ij}^{T} {Ω}_{ij} {A}_{ij}$ 
 - $\bar{H}_{jj} {+}{=} {B}_{ij}^{T} {Ω}_{ij} {B}_{ij}$



In [ ]:
import numpy as np
import scipy.io
import re
import matplotlib.pyplot as plt

def open_and_assing(file_path):
    """
    Opens the file.
    
    params:
      file_path (string) :  matlab(.mat extension) file path. 
      
    returns:
      x (array) : state vector.
      edges (array) :  measurements, information matrices, edges.
      dimension (array) :  to visualize.
      ofsett (array) :  to visualize.

    """

    data = scipy.io.loadmat(file_path)["g"][0][0]
    x = data[0]
    edges = data[1]
    
    # to visualize
    list1 = np.array(re.findall('\d+', str(data[2][0][0])),dtype = np.int16)[2:]
    dimension = list1[1::2] # Elements from list1 starting from 1 iterating by 2
    offset = list1[::2]  # Elements from list1 starting from 0 iterating by 2
    
    return x, edges, dimension, offset


def v2t(v):
    """
    Computes the homogeneous transform matrix A of the pose vector v.
    
    params:
      v (array) : pose vector.
      
    returns:
      A (array) : transform matrix.
      
    """

    c = np.cos(v[2][0])
    s = np.sin(v[2][0])
    A = np.array([[c, -s, v[0][0]], 
                  [s, c, v[1][0]], 
                  [0, 0, 1]], dtype='float')
    return A
    
    
def t2v(A):
    """
    Computes the pose vector v from a homogeneous transform matrix A.
        
    params:
      A (array) : transform matrix.
      
    returns:
      v (array) : pose vector.
    """
    
    v = np.array([A[0][2], A[1][2], 
                  np.arctan2(A[1][0], 
                             A[0][0])], dtype='float')
    return v

def plot_graph(x, iteration):
    """
    To visualize.
    """
    # initialize figure
    plt.figure(1)
    plt.clf()
    p = []
    l = []

    for i, dim in enumerate(dimension):
        if dim == 3:
            p.append(offset[i])

        elif int(dim) == 2:
            l.append(offset[i])
            
    p = np.array(p).reshape(len(p),1)
    l = np.array(l).reshape(len(l),1)

    if len(l) > 0:
        landmarkIdxX = [int(i) for i in l]
        landmarkIdxY = [int(i + 1) for i in l]
        plt.plot(x[landmarkIdxX], x[landmarkIdxY], 'or', markersize=4)      
        
    if len(p) > 0:
        pIdxX = [int(i ) for i in p]
        pIdxY = [int(i + 1) for i in p]
        plt.plot(x[pIdxX], x[pIdxY], 'xb', markersize=3)
        
    plt.draw()
    plt.pause(1)
    


`compute_global_error ;`

- ${F(x)}= \sum_{i} {e}_{i}{(x)}^T  {Ω}_{i} {e}_{i}{(x)} $



- Error function for pose to pose:

 - ${e}_{ij} ({x}_{i},{x}_{j}) = $ t2v $({Z}_{ij}^{-1} ({X}_{i}^{-1} {X}_{j}))$

   - ${Z}_{ij}^{-1}$ = measurement (How ${x}_{j}$ is seen by ${x}_{i}$)

   - ${X}_{i}^{-1} {X}_{j}$ = ${x}_{i}$ referenced w.r.t ${x}_{j}$ (How ${x}_{j}$ is seen by ${x}_{i}$)

   - t2v, v2t -> forward, backward mapping between homegenous coordinate and x, y, theta.

   -  ${Z}_{ij} = {X}_{i}^{-1} {X}_{j} $ -> error will be zero.
   
   
   
- Error function for landmark to pose (x-y sensor):

 - ${e}_{ij} ({x}_{i},{x}_{j}) = \hat{z}_{ij} - {z}_{ij} = {R}_{i}^{T}({x}_{j} - {t}_{i}) - {z}_{ij}$
 
   - ${x}_{i} $ -> robot pose - 3D

   - ${x}_{j} $ -> landmark pose - 2D

   - ${t}_{i} $ -> robot translation vector

Note  : (x-y) sensor tells us where the landmark is in an relative xy coordinate frame given the robot's pose.

Note2 : We need at least 2 observations to solve the problem.



In [ ]:
def compute_global_error(x, edges):
    """
    Computes the global error. 
    
    params:
      x (array) : state vector.
      edges (array) :  measurements, information matrices, edges.   
      
    returns:
      F(x) (numpy.float64) : the global error.
    """
    
    Fx = 0
    for edge in edges:
        edge = edge[0]
        
        # pose-pose constraint
        if(edge[0] == 'P'):
            x1 = x[int(edge[5])-1:int(edge[5])+2] # the first robot pose
            x2 = x[int(edge[6])-1:int(edge[6])+2] # the second robot pose
            z12 = edge[3] # the measurement 
            info12 = edge[4] # the information matrix corresponding to the measurement
            
            # compute the error of the constraint and add it to Fx.
            es = t2v(np.linalg.inv(v2t(z12)) @ np.linalg.inv(v2t(x1)) @ v2t(x2))
            Fx += es.T @ info12 @ es 
            
        # pose-landmark constraint   
        elif (edge[0] == 'L'):
            xr = x[int(edge[5])-1:int(edge[5])+2] # the robot pose
            l = x[int(edge[6])-1:int(edge[6])+1] # the landmark
            z = edge[3] # the measurement 
            info = edge[4] # the information matrix corresponding to the measurement
            
            es = np.linalg.inv(v2t(xr)) @ np.array([l[0][0], l[1][0], 1]) - np.array([z[0][0], z[1][0], 1])
            es = es[:2]
            Fx += es.T @ info @ es    

    return Fx

In [ ]:
def linearize_pose_pose_constraint(x1, x2, z):
    """
    Computes the error of a pose-pose constraint.
    
    params:
        x1 (array): 3x1 vector (x, y, theta) of the first robot pose.
        x2 (array): 3x1 vector (x, y, theta) of the second robot pose.
        z  (array): 3x1 vector (x, y, theta) of the measurement.
    
    returns:
         e (array): 3x1 error of the constraint.
         A (array): 3x3 Jacobian w.r.t. x1.
         B (array): 3x3 Jacobian w.r.t. x2.
    """

    # compute the transformation from the vectors  
    x_i = v2t(x1) # first pose
    x_j = v2t(x2) # second pose
    z_ij = v2t(z) # measurement between first pose and second pose
    
    R_i = x_i[0:2, 0:2]  # 2*2 rotation [[c(theta_i), -s(theta_i)], [s(theta_i),c(theta_i)]]
    R_ij = z_ij[0:2, 0:2]  # 2*2 rotation [[c(theta_ij), -s(theta_ij)], [s(theta_ij),c(theta_ij)]]
    ct_i = np.cos(x1[2][0])
    st_i = np.sin(x1[2][0])
    dRdT_i = np.array([[-st_i, ct_i], [-ct_i, -st_i]])
  
    # compute the error
    e = t2v(np.linalg.inv(z_ij) @ (np.linalg.inv(x_i) @ x_j))


    # compute the Jacobians of the error

    A = np.vstack((np.hstack((-R_ij.T @ R_i.T, np.array((R_ij.T @ dRdT_i) @ (x2[0:2] - x1[0:2])))), [0,0,-1]))

    B = np.vstack((np.hstack((R_ij.T @ R_i.T, [[0],[0]])), [0,0,1]))

    return e, A, B

In [ ]:
def linearize_pose_landmark_constraint(x, l, z):
    """
    Computes the error of a pose-landmark constraint.
    
    params:
        x (array): 3x1 vector (x, y, theta) of the robot pose.
        l (array): 2x1 vector (x, y) of the landmark
        z (array): 2x1 vector (x, y) of the measurement, the position of the landmark in \
        the coordinate frame of the robot given by the vector x.
    
    returns:
         e (array): 2x1 error of the constraint.
         A (array): 2x3 Jacobian w.r.t. x.
         B (array): 2x3 Jacobian w.r.t. l.
    """

    # compute the error 
    xt = v2t(x)
    e = np.linalg.inv(xt) @ [l[0][0], l[1][0], 1] - [z[0][0], z[1][0], 1]
    e = e[0:2]
    
    # compute the Jacobians of the error
    Ri = xt[0:2, 0:2]
  
    ct_i = np.cos(x[2][0])
    st_i = np.sin(x[2][0])
    dRdT_i = np.array([[-st_i, ct_i], [-ct_i, -st_i]])

 
    A = np.hstack((-Ri.T, dRdT_i @ (l - x[0:2]))) # 2*3
    B = Ri.T  # 2*2

    return e, A, B

In [ ]:
def linearize_and_solve(x, edges):
    """
    Solves the least squares problem by linearizing the constraints.
        
    params:
      x (array) : state vector.
      edges (array) :  measurements, information matrices, edges.   
      
    returns:
      dx (array) : change in the unknowns x.
      
    """
    needToAddPrior = True
    # allocate the sparse H and the vector b
    H = np.zeros((len(x), len(x)))
    b = np.zeros(len(x))
    
    for edge in edges:
        edge = edge[0]
        if(edge[0] == 'P'):
            x1 = x[int(edge[5])-1:int(edge[5])+2] # the first robot pose
            x2 = x[int(edge[6])-1:int(edge[6])+2] # the second robot pose
            z12 = edge[3] # the measurement 
            info12 = edge[4] # the information matrix corresponding to the measurement
            
            # computing the error and the Jacobians
            e, A, B = linearize_pose_pose_constraint(x1, x2, z12)
            
            # compute and add the term introduced by the pose-pose constraint to H and b
            b_i  = -(e.T @ info12 @ A).T
            b_j  = -(e.T @ info12 @ B).T
            H_ii = A.T @ info12 @ A
            H_ij = A.T @ info12 @ B 
            H_jj = B.T @ info12 @ B 

            # Adding them to the H and b
            
            H[int(edge[5])-1:int(edge[5])+2, int(edge[5])-1:int(edge[5])+2] += H_ii
            H[int(edge[5])-1:int(edge[5])+2, int(edge[6])-1:int(edge[6])+2] += H_ij
            H[int(edge[6])-1:int(edge[6])+2, int(edge[5])-1:int(edge[5])+2] += H_ij.T
            H[int(edge[6])-1:int(edge[6])+2, int(edge[6])-1:int(edge[6])+2] += H_jj
            b[int(edge[5])-1:int(edge[5])+2] =  b[int(edge[5])-1:int(edge[5])+2] + b_i;
            b[int(edge[6])-1:int(edge[6])+2] =  b[int(edge[6])-1:int(edge[6])+2] + b_j;

            if (needToAddPrior):
                # This fixes one node to remain at its current location.
                H[int(edge[5])-1:int(edge[5])+2, int(edge[5])-1:int(edge[5])+2] += 1000*np.eye(3);
                needToAddPrior = False


        elif (edge[0] == 'L'):            
            xr = x[int(edge[5])-1:int(edge[5])+2] # the robot pose
            l = x[int(edge[6])-1:int(edge[6])+1] # the landmark
            z = edge[3] # the measurement 
            info = edge[4] # the information matrix corresponding to the measurement
            
            # computing the error and the Jacobians
            e, A, B = linearize_pose_landmark_constraint(xr, l, z)
            
            # compute and add the term introduced by the pose-landmark constraint to H and b
            b_i  = -(e.T @ info @ A).T
            b_j  = -(e.T @ info @ B).T
            H_ii = A.T @ info @ A
            H_ij = A.T @ info @ B 
            H_jj = B.T @ info @ B 

            H[int(edge[5])-1:int(edge[5])+2, int(edge[5])-1:int(edge[5])+2] += H_ii
            H[int(edge[5])-1:int(edge[5])+2, int(edge[6])-1:int(edge[6])+1] += H_ij
            H[int(edge[6])-1:int(edge[6])+1, int(edge[5])-1:int(edge[5])+2] += H_ij.T
            H[int(edge[6])-1:int(edge[6])+1, int(edge[6])-1:int(edge[6])+1] += H_jj
            b[int(edge[5])-1:int(edge[5])+2] = b[int(edge[5])-1:int(edge[5])+2] +  b_i;
            b[int(edge[6])-1:int(edge[6])+1] = b[int(edge[6])-1:int(edge[6])+1] +  b_j;
            

    # solve the linear system , dx = H\b 
    b = b.reshape(b.shape[0],1)
    dx = np.linalg.solve(H,b)

    return dx

In [ ]:
# x, edges, dimension, offset = open_and_assing('data/intel.mat') # Real-world dataset
 x, edges, dimension, offset = open_and_assing('data/simulation-pose-landmark.mat') # Simulation dataset
# x, edges, dimension, offset = open_and_assing('data/dlr.mat') # Real-world dataset

# the number of iterations
numIterations = 100

# maximum allowed dx
EPSILON = 10**-4

# error
err = 0

print('Initial error', compute_global_error(x, edges))

# iterate
for i in range(numIterations):
    print('Performing iteration', i)
    # compute the incremental update dx of the state vector
    dx = (linearize_and_solve(x, edges)).reshape(x.shape)
    # apply the solution to the state vector x
    x = x + dx
    # plot the current state of the graph
    plot_graph(x, numIterations)

    # compute the global error given the current graph configuration
    err = compute_global_error(x, edges)
    # print current error
    print('Current error', err)

    # termination criterion
    if max(abs(dx)) < EPSILON:
        print('Converged!!')
        break

print('Done!\nFinal error', err)

#### Extra Informations : ####

`Bearing Only Observation :`
- A landmark is 2D point.
- The robot observe only only the bearing toward the landmark.
- Bearing sensor tells us which direction we can see a landmark. 
- Expected Observation = $\hat{z}_{ij} ({x}_{i},{x}_{j}) = atan\frac{({x}_{j}- {t}_{i}).{y}} {({x}_{j} - {t}_{i}).{x}} - {\theta}_{i}$

- ${x}_{i}$ -> robot pose
- ${x}_{j}$ -> landmark 
- $atan\frac{({x}_{j}- {t}_{i}).{y}} {({x}_{j} - {t}_{i}).{x}}$ -> robot- landmark angle (robot pose with respect to the landmark)
- ${\theta}_{i}$ -> robot orientation
- $\hat{z}_{ij} ({x}_{i},{x}_{j})$ is one-dimensional function. It is two-dimensional for xy sensor.
- Error Function = ${e}_{ij} ({x}_{i},{x}_{j}) = atan\frac{({x}_{j}- {t}_{i}).{y}} {({x}_{j} - {t}_{i}).{x}} - {\theta}_{i}- {z}_{j}$

Note: We need at least 3 observations to solve the problem.



`Under-Determined Systems :`

- No guarantee for a full rank system
 - Landmarks may be observed only once.
 - Robot might have no odometry.
- We can still deal with these situations by adding a "damping" factor to H
- Instead of solving $H\Delta x = -b$, we solve $(H + \lambda I)\Delta x = -b$. Now it is full rank. (Simplified Levenberg Marquardt)
- The damping factor $\lambda I$ makes the system positive definite.


`Bundle Adjustment :`
- 3D reconstruction based on images taken at different viewpoints.
- Minimizes the reprojection error in the 2D image plane.
- No notation of odometry (pose-pose)
- Often uses Levenberg Marquardt